In [1]:
dbutils.fs.mkdirs("/mnt/amazon_review_data")

Out[1]: True

In [2]:
%sh wget wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz -O /tmp/cellphone_reviews.json.gz

wget: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /databricks/python/lib/libssl.so.1.0.0: no version information available (required by wget)
wget: /databricks/python/lib/libssl.so.1.0.0: no version information available (required by wget)
--2019-03-17 12:36:12-- http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2019-03-17 12:36:12-- http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45409631 (43M) [application/x-gzip]
Saving to: ‘/tmp/cellphone_reviews.json.gz’

 0K .......... .......... .......... .......... .......... 0% 1020K 43s
 50K .......... .......... .......... .......... .......... 0% 1024K 43s
 100K .......... .......... .......... .......... .......... 0% 2.00M 36s
 150K .......... .......... .......... .......... .......... 0% 1.99M 32s
 200K .......... .......... .......... .......... .......... 0% 2.04M 30s
 250K .......... .......... .......... .......... .......... 0% 60.2M 25s
 300K .......... .......... .......... .......... .......... 0% 2.04M 25s
 350K .......... .......... .......... .......... .......... 0% 90.6M 22s
 400K .......... .......... .......... .......... .......... 1% 2.04M 21s
 450K .......... .......... .......... .......... .......... 1% 78.4M 19s
 500K .......... .......... .......... .......... .......... 1% 2.06M 19s
 550K .......... .......... .......... .......... .......... 1% 64.4M 18s
 600K .......... .......... .......... .......... .......... 1% 143M 17s
 650K .......... .......... .......... .......... .......... 1% 2.08M 17s
 700K .......... .......... .......... .......... .......... 1% 66.9M 16s
 750K .......... .......... .......... .......... .......... 1% 156M 15s
 800K .......... .......... .......... .......... .......... 1% 2.10M 15s
 850K .......... .......... .......... .......... .......... 2% 48.1M 14s
 900K .......... .......... .......... .......... .......... 2% 94.7M 13s
 950K .......... .......... .......... .......... .......... 2% 95.6M 13s
 1000K .......... .......... .......... .......... .......... 2% 2.15M 13s
 1050K .......... .......... .......... .......... .......... 2% 54.4M 13s
 1100K .......... .......... .......... .......... .......... 2% 93.8M 12s
 1150K .......... .......... .......... .......... .......... 2% 96.9M 12s
 1200K .......... .......... .......... .......... .......... 2% 2.19M 12s
 1250K .......... .......... .......... .......... .......... 2% 57.6M 11s
 1300K .......... .......... .......... .......... .......... 3% 76.8M 11s
 1350K .......... .......... .......... .......... .......... 3% 80.2M 11s
 1400K .......... .......... .......... .......... .......... 3% 123M 10s
 1450K .......... .......... .......... .......... .......... 3% 2.21M 10s
 1500K .......... .......... .......... .......... .......... 3% 75.8M 10s
 1550K .......... .......... .......... .......... .......... 3% 58.5M 10s
 1600K .......... .......... .......... .......... .......... 3% 81.8M 10s
 1650K .......... .......... .......... .......... .......... 3% 90.7M 9s
 1700K .......... .......... .......... .......... .......... 3% 2.06M 10s
 1750K .......... .......... .......... .......... .......... 4% 40.3M 9s
 1800K .......... .......... .......... .......... .......... 4% 88.7M 9s
 1850K .......... .......... .......... .......... .......... 4% 95.3M 9s
 1900K .......... .......... .......... .......... .......... 4% 73.0M 9s
 1950K .......... .......... .......... .......... .......... 4% 149M 8s
 2000K .......... .......... .......... .......... .......... 4% 2.34M 9s
 2050K .......... .......... .......... .......... .......... 4% 45.4M 8s
 2100K .......... .......... .

In [3]:
dbutils.fs.mv("file:///tmp/cellphone_reviews.json.gz","/mnt/amazon_review_data/cellphone_reviews.json.gz")

Out[4]: True

In [4]:
%fs ls '/mnt/amazon_review_data/cellphone_reviews.json.gz'

path,name,size
dbfs:/mnt/amazon_review_data/cellphone_reviews.json.gz,cellphone_reviews.json.gz,45409631


In [5]:
#convert data files to strict json
import json
import gzip

def parse(path):
  g = gzip.open(path,'r')
  for l in g:
    yield json.dumps(eval(l))
f = open("/tmp/cellphone_reviews.json","w")
for l in parse("/dbfs/mnt/amazon_review_data/cellphone_reviews.json.gz"):
  f.write(l + '\n')


In [6]:
dbutils.fs.mv("file:///tmp/cellphone_reviews.json","/mnt/amazon_review_data/cellphone_reviews.json")

Out[14]: True

In [7]:
%fs ls '/mnt/amazon_review_data/'



path,name,size
dbfs:/mnt/amazon_review_data/cellphone_reviews.json,cellphone_reviews.json,141685977
dbfs:/mnt/amazon_review_data/cellphone_reviews.json.gz,cellphone_reviews.json.gz,45409631


In [8]:
amazon_reviews_df = spark.read.format('json').options(header='true',inferSchema='true').load('/mnt/amazon_review_data/cellphone_reviews.json')
amazon_reviews_df.count()

Out[15]: 194432

In [9]:
display(amazon_reviews_df)

asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
120401325X,"List(0, 0)",4.0,They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again,"05 21, 2014",A30TL5EWN6DFXT,christina,Looks Good,1400630400
120401325X,"List(0, 0)",5.0,These stickers work like the review says they do. They stick on great and they stay on the phone. They are super stylish and I can share them with my sister. :),"01 14, 2014",ASY55RVNIL0UD,emily l.,Really great product.,1389657600
120401325X,"List(0, 0)",5.0,These are awesome and make my phone look so stylish! I have only used one so far and have had it on for almost a year! CAN YOU BELIEVE THAT! ONE YEAR!! Great quality!,"06 26, 2014",A2TMXE2AFO7ONB,Erica,LOVE LOVE LOVE,1403740800
120401325X,"List(4, 4)",4.0,"Item arrived in great time and was in perfect condition. However, I ordered these buttons because they were a great deal and included a FREE screen protector. I never received one. Though its not a big deal, it would've been nice to get it since they claim it comes with one.","10 21, 2013",AWJ0WZQYMYFQ4,JM,Cute!,1382313600
120401325X,"List(2, 3)",5.0,"awesome! stays on, and looks great. can be used on multiple apple products. especially having nails, it helps to have an elevated key.","02 3, 2013",ATX7CZYFXI1KW,patrice m rogoza,leopard home button sticker for iphone 4s,1359849600
120401325X,"List(1, 2)",3.0,These make using the home button easy. My daughter and I both like them. I would purchase them again. Well worth the price.,"10 12, 2013",APX47D16JOP7H,RLH,Cute,1381536000
120401325X,"List(0, 0)",5.0,Came just as described.. It doesn't come unstuck and its cute! People ask where I got them from & it's great when driving.,"08 22, 2013",A1JVVYYO7G56DS,Tyler Evans,best thing ever..,1377129600
3998899561,"List(1, 2)",1.0,it worked for the first week then it only charge my phone to 20%. it is a waste of money.,"11 21, 2013",A6FGO4TBZ3QFZ,Abdullah Albyati,not a good Idea,1384992000
3998899561,"List(2, 3)",5.0,"Good case, solid build. Protects phone all around with good access to buttons. Battery charges with full battery lasts me a full day. I usually leave my house around 7am and return at 10pm. I'm glad that it lasts from start to end. 5/5","09 25, 2013",A2JWEDW5FSVB0F,Adam,Solid Case,1380067200
3998899561,"List(1, 1)",5.0,"This is a fantastic case. Very stylish and protects my phone. Easy access to all buttons and features, without any loss of phone reception. But most importantly, it double power, just as promised. Great buy","04 3, 2014",A8AJS1DW7L3JJ,Agata Majchrzak,Perfect Case,1396483200


In [10]:
amazon_reviews_df.printSchema()

root
-- asin: string (nullable = true)
-- helpful: array (nullable = true)
 |-- element: long (containsNull = true)
-- overall: double (nullable = true)
-- reviewText: string (nullable = true)
-- reviewTime: string (nullable = true)
-- reviewerID: string (nullable = true)
-- reviewerName: string (nullable = true)
-- summary: string (nullable = true)
-- unixReviewTime: long (nullable = true)

In [11]:
amazon_reviews_df.columns

Out[18]: 
['asin',
 'helpful',
 'overall',
 'reviewText',
 'reviewTime',
 'reviewerID',
 'reviewerName',
 'summary',
 'unixReviewTime']

In [12]:
#save as parquet file format:

dbutils.fs.mkdirs("/mnt/parquet/amazon_reviews_data")
amazon_reviews_df.write.mode("overwrite").format("parquet").save("/mnt/parquet/amazon_reviews_data")


In [13]:
%fs ls /mnt/parquet/amazon_reviews_data

path,name,size
dbfs:/mnt/parquet/amazon_reviews_data/_SUCCESS,_SUCCESS,0
dbfs:/mnt/parquet/amazon_reviews_data/_committed_3908239577296777519,_committed_3908239577296777519,816
dbfs:/mnt/parquet/amazon_reviews_data/_started_3908239577296777519,_started_3908239577296777519,0
dbfs:/mnt/parquet/amazon_reviews_data/part-00000-tid-3908239577296777519-92ca9621-b165-4932-a048-bccb5b57b3e4-18-1-c000.snappy.parquet,part-00000-tid-3908239577296777519-92ca9621-b165-4932-a048-bccb5b57b3e4-18-1-c000.snappy.parquet,8554611
dbfs:/mnt/parquet/amazon_reviews_data/part-00001-tid-3908239577296777519-92ca9621-b165-4932-a048-bccb5b57b3e4-19-1-c000.snappy.parquet,part-00001-tid-3908239577296777519-92ca9621-b165-4932-a048-bccb5b57b3e4-19-1-c000.snappy.parquet,8262724
dbfs:/mnt/parquet/amazon_reviews_data/part-00002-tid-3908239577296777519-92ca9621-b165-4932-a048-bccb5b57b3e4-20-1-c000.snappy.parquet,part-00002-tid-3908239577296777519-92ca9621-b165-4932-a048-bccb5b57b3e4-20-1-c000.snappy.parquet,8195900
dbfs:/mnt/parquet/amazon_reviews_data/part-00003-tid-3908239577296777519-92ca9621-b165-4932-a048-bccb5b57b3e4-21-1-c000.snappy.parquet,part-00003-tid-3908239577296777519-92ca9621-b165-4932-a048-bccb5b57b3e4-21-1-c000.snappy.parquet,7836999
dbfs:/mnt/parquet/amazon_reviews_data/part-00004-tid-3908239577296777519-92ca9621-b165-4932-a048-bccb5b57b3e4-22-1-c000.snappy.parquet,part-00004-tid-3908239577296777519-92ca9621-b165-4932-a048-bccb5b57b3e4-22-1-c000.snappy.parquet,8045113
dbfs:/mnt/parquet/amazon_reviews_data/part-00005-tid-3908239577296777519-92ca9621-b165-4932-a048-bccb5b57b3e4-23-1-c000.snappy.parquet,part-00005-tid-3908239577296777519-92ca9621-b165-4932-a048-bccb5b57b3e4-23-1-c000.snappy.parquet,8323964
dbfs:/mnt/parquet/amazon_reviews_data/part-00006-tid-3908239577296777519-92ca9621-b165-4932-a048-bccb5b57b3e4-24-1-c000.snappy.parquet,part-00006-tid-3908239577296777519-92ca9621-b165-4932-a048-bccb5b57b3e4-24-1-c000.snappy.parquet,8557938


In [14]:
%sql
CREATE TEMPORARY TABLE amazon_reviews
USING PARQUET
OPTIONS (path "/mnt/parquet/amazon_reviews_data")

In [15]:
%sql describe amazon_reviews

col_name,data_type,comment
asin,string,null
helpful,array,null
overall,double,null
reviewText,string,null
reviewTime,string,null
reviewerID,string,null
reviewerName,string,null
summary,string,null
unixReviewTime,bigint,null


In [16]:
%sql
select count(*) from amazon_reviews

count(1)
194432


In [17]:
%sql select * from amazon_reviews limit 100

asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
B00C80S4W6,"List(2, 2)",5.0,"This is a great quality protector that protects your screen.This screen protector is a great way to protect your phone from scratches, nicks, and dust. Even before installing the screen protectors, I knew it was good quality just from the feel in the hand. These protectors are more of a second screen rather than just a sheet of a plastic.Like all protectors they can be tricky to install, so be sure to read the instructions on the back of the package. After reading the instructions they go on like a piece of cake. The first step is to clean the screen with the enclosed lint free cloth. I find it best to position the screen around the camera and sensor first.While applying the screen protector use the smoothing card (squeegee) to firmly affix it to the screen. There may be some small air bubbles, they will disappear within a day or two. Once all of these steps you have completed, you will have a phone screen that will safely protected.These screen protectors are step above the rest because of their special features. Which are: anti-fingerprint; scratch resistant; anti-bubble; super clear; oil-resistant. This lets me know they have the customer in mind.The best part of this kit beside the great screen protector, is the smoothing card (squeegee) that is made out of plastic and not cardboard like the other cheap ones on the market.CONTENTS:Illumishield Screen Protectors (3x)Smoothing CardLint-Free Polishing ClothInstructions (Back Of Package)I was supplied a sample for the purpose of review.","03 10, 2014",A18U49406IPPIJ,Deon,Great Way To Protect Your Note 2,1394409600
B00C80S4W6,"List(2, 2)",5.0,"Very,very well made. Easy application. Perfect fit. I bought the same for my ipad1&2. Product was as promised and provided glass like feel. Superior!!","07 5, 2013",ADKXBC6D7XOYE,Desdemona Sesay,BEST SCREEN PROTECTOR !!,1372982400
B00C80S4W6,"List(0, 1)",1.0,"These are the absolute worst screen protecter I have ever had. It's cheap, feels like cheap plastic not GLASS like stated. All three came with scratches on them. Impossible to get protective film off it without damaging them. Worst experience ever BUY TEMPERED GLASS.","05 10, 2014",A35ZDD9B5ZBUHL,Justin Shorey,DO NOT BUY. LISTON TO ME,1399680000
B00C80S4W6,"List(1, 1)",5.0,Best choiceGood quality and fit protectionDelivery was fastSuper clear and anti finger tooGood choice if u want something special,"12 16, 2013",A1M2QRD34Q21H2,MoHaMeD,Best choice and good quality,1387152000
B00C80S4W6,"List(0, 1)",2.0,They go on very easily. Pretty nice fit. Any if any bubbles rub right out. Looks great at first. They do smear with fingerprints though. Not at first but soon I found myself wiping my screen off as much as without a screen protector. The worst part was after a week it started pealing up on the top edge.,"10 7, 2013",A2IESVWM2WYXL7,Plc,temporary,1381104000
B00C80S4W6,"List(1, 1)",5.0,This is one of the best screen protectors that you can buy. I have used this brand on my Galaxy Note 3 and now I am using it on my Windows Phone 1520.,"05 18, 2014",A2Y9TKEO052TES,Sturghill,excellent screen protector,1400371200
B00C80S4W6,"List(0, 0)",2.0,"This review gets 2 stars for 1 reason only.The note 3 protectors, No complaints.But the gear protectors are a joke!!!!!!!!! Anyone who has a gear watch knows the whole face is made of glass. These protectors were little cut out squares that would go in the middle of the glass and cover just the screen part that lights up and black when off. What the?? That's just retarded. It needs to cover the WHOLE surface area of the glass or else what's the point?? It will fall off easy and doesn't provide full protection. Went into trash....","07 21, 2014",AXIYN1SRSJ9L0,Toni,What a joke; the gear protectors. Save your $,1405900800
B00C80S4W6,"List(0, 1)",3.0,"I purchased this screen protector for my Galaxy Gear to protect the screen. The product wor

In [18]:
%sql select overall as count from amazon_reviews group by overall order by count(*) desc limit 25

count
5.0
4.0
3.0
1.0
2.0


In [19]:
%sql 
select overall , count(1) as count
from amazon_reviews group by overall order by overall desc

overall,count
5.0,108658
4.0,39992
3.0,21439
2.0,11064
1.0,13279


In [20]:
from pyspark.sql import functions as F
def countWordsInEachSentences(array):
    return [len(x.split()) for x in array]

countWordsSentences = F.udf(lambda x: countWordsInEachSentences(x.split('. ')))
amazon_reviews_df.withColumn("word_count", countWordsSentences(amazon_reviews_df['reviewText'])).show(truncate=False)

+----------+--------+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+--------------+---------------------------------+----------------------------------------------------------------+--------------+-----------------------------------+
asin |helpful |overall|reviewText |reviewTime |reviewerID |reviewerName |summary |unixReviewTime|word_count |
+----------+--------+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+--------------+---------------------------------+----------------------------------------------------------------+--------------+-----------------------------------+
120401325X|[0, 0] |4.0 |They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again |05 21, 2014|A30TL5EWN6DFXT|christina |Looks Good |1400630400 |[28, 9] |
120401325X|[0, 0] |5.0 |These stickers work like the review says they do. They stick on great and they stay on the phone. They are super stylish and I can share them with my sister. :) |01 14, 2014|ASY55RVNIL0UD |emily l. |Really great product. |1389657600 |[9, 10, 12, 1] |
120401325X|[0, 0] |5.0 |These are awesome and make my phone look so stylish! I have only used one so far and have had it on for almost a year! CAN YOU BELIEVE THAT! ONE YEAR!! Great quality! |06 26, 2014|A2TMXE2AFO7ONB|Erica |LOVE LOVE LOVE |1403740800 |[34] |
120401325X|[4, 4] |4.0 |Item arrived in great time and was in perfect condition. However, I ordered these buttons because they were a great d

In [21]:
splitSample = spark.table('amazon_reviews').randomSplit([.70,.30], 123)
trainingData = splitSample[0]
testData = splitSample[1]

In [22]:
from pyspark.ml import *
from pyspark.ml.feature import *
from pyspark.ml.feature import Bucketizer
from pyspark.ml.classification import *
from pyspark.ml.tuning import *
from pyspark.ml.evaluation import *
from pyspark.ml.regression import *

splits = [-float("inf"),4.5, float("inf")]
tok = Tokenizer(inputCol = "reviewText", outputCol = "words")
bucket = Bucketizer(splits=splits, inputCol="overall", outputCol = "label")


hashTF = HashingTF(inputCol = tok.getOutputCol(), numFeatures = 10000 ,outputCol ="features")

lr = LogisticRegression(maxIter = 10, regParam= 0.0001, elasticNetParam =1.0)

pipeline = Pipeline(stages=[bucket,tok,hashTF,lr])

In [23]:
model = pipeline.fit(trainingData)

In [24]:
predictions = model.transform(testData)
predictions.createOrReplaceTempView('tmp_predictions')

In [25]:
%sql
select label, prediction,probability, overall, reviewText,helpful
from tmp_predictions 
where reviewText like "%return%"

label,prediction,probability,overall,reviewText,helpful
0.0,0.0,"List(1, 2, List(), List(0.7721109156242334, 0.22788908437576663))",2.0,"Was not very happy with the quality of these devices, so I returned them without every using them, and purchased a Mophie cover. Not sure I would have not liked them, just didn't seem as high quality as Mophie.","List(0, 0)"
0.0,0.0,"List(1, 2, List(), List(0.7147834912520842, 0.2852165087479159))",4.0,"Although I was tempted to keep the Commuter case for my S4, I ended up returning it in favor of the Incipio DualPro Shine case. I personally didn't enjoy the smooth back with curve built into it as I felt like it was hard to hold onto and slipped easily. I would not doubt that the protection for this case is spectacular as OtterBox is usually associated with good products. The Commuter case is comparatively more expensive when placed against similar cases.","List(0, 0)"
0.0,0.0,"List(1, 2, List(), List(0.933026893364202, 0.06697310663579806))",1.0,It Looked red and gray to me. When I got it it was pink and gray. My mistake though. They did not leave a return receipt in the package. I did not know how to return it.,"List(0, 4)"
0.0,0.0,"List(1, 2, List(), List(0.9988702211820992, 0.0011297788179007644))",4.0,"at first i was unhappy with this case. the first few weeks, it used to blank the screen after a call or during a call and nothing would bring the screen back. i figured out it was the case once when i started to remove it to try to take the battery out. then the screen would come to life and i could end the call. one time someone butt dialed me and i could not hang up on them for 15 mins until i started to take the case off.anyway, it's not doing it anymore and i have had this for 6 months now. every once in awhile it does it, but i can live with it.i hate the stupid flaps for the mic jack and the charger, so i cut them off with thin snips and a rotozip since they have steel wire in them and just a knife would not work.also, i took off the screen protector. it was too thick and was making pressing buttons very hard.i had printed a return label to ship it back, but i am satisfied that i did not return it. it's a bit pricey though. i didn't like that part.i have dropped it many times, been in attics, basements, garages, bumped, everything... no issues.the clip has held up well. it never comes off the clip if installed correctly.","List(0, 0)"
1.0,0.0,"List(1, 2, List(), List(0.8030058258910435, 0.19699417410895637))",5.0,I'm sure this is a great product. It seemed well made. I did return it but only because I returned the phone it was designed to protect. I've had otterbox products in the past on prior phones & they were well worth their slightly higher upfront cost. I wish they made one for the phone I now have!,"List(0, 0)"
1.0,0.0,"List(1, 2, List(), List(0.9416341770828943, 0.05836582291710576))",5.0,"Very happy with the case. Protects the phone well, the only thing is that if I want to replace the battery, I have to remove that case which can be a pain. The case is four pieces with the holster, so you have to be in a spot where you can do it and not drop the pieces of the case. But well worth the trouble of taking it apart and putting it back together. The only problem I had was I had to return the first one because the clip on the holster when I knocked into the wall. the second one has been fine. Oh, and there were some comments about the phone not locking into the holster securely, as well it not locking in with the screen facing outward, which is necessary if you want to use the stand feature of the holster. I have NO problem doing either of these. Simple trick is to be careful, listen for the click of it locking in and give it a tug to make sure. Good case.","List(0, 0)"
1.0,0.0,"List(1, 2, List(), List(0.934503715361796, 0.06549628463820399))",5.0,"+++++++++++++++++++++++++++++++++++January 2014 update: While this is a good product, I think it was built for those the do NOT change